In [48]:
import pandas as pd
from collections import defaultdict, Counter
import numpy as np
import re

In [49]:
data = pd.read_csv('data_preprocessed.csv')
data

,original_text,processed_text
0,Mending plonga plong Tinimbang pinter go minteri,mending bingung plong timbang pintar go minteri
1,Hidup amin Raiso bapa repormasi,hidup raiso bapa repormasi
2,Mantaappps... Btlllll pak...,mantaps btl
3,Yang menghina itu lambemu min,menghina itu lambe min
4,Amin rais manusia paling redah pikiranya..peng...,manusia redah pikiranya abdi hawa nafsu iri de...
...,...,...
18770,@kompascom Tapi aiman bertanya ke pengacaranya...,aiman acara ijazah edar pegang arti ini lindun...
18771,@muannas_alaidid Yg jelas omongannya Jokowi pa...,omong palsu titik
18772,@CakKhum depresi si om Jokowi,depresi si om
18773,@TriSartopo @ch_chotimah2 @jokowi Mana berani ...,berani tatut hm maklum curut berani cuap aja y...


In [50]:
cnt = Counter()
for text in data['processed_text'].values:
    for word in text.split():
        cnt[word] += 1
cnt.most_common(1000)
temp = pd.DataFrame(cnt.most_common(1000))
temp.columns=['word', 'count']
temp

,word,count
0,orang,4315
1,ini,4073
2,kakek,3929
3,kamu,3209
4,tua,2970
...,...,...
995,ngurusi,18
996,tuk,18
997,batu,18
998,q,18


In [51]:
## Proses Labelling menggunakan Rule Based System
CYBERBULLYING_RULES = {
    'berbahaya': {
        'weight': 5.0,  # PALING TINGGI - berbahaya
        'keywords': [
            'bunuh', 'mati', 'modar', 'mampus', 'habisi', 'tangkap', 'pukul',
            'hajar', 'libas', 'samplok', 'gebuk', 'sikat', 'tendang', 'lempar',
            'bakar', 'potong', 'tikam', 'tembak', 'hancur', 'musnahkan'
        ]
    },

    'serangan_langsung': {
        'weight': 4.0,  # TINGGI - serangan langsung
        'keywords': [
            'bodoh', 'gila', 'tolol', 'dungu', 'goblok', 'idiot', 'bego', 'lemot',
            'bangsat', 'bajingan', 'kurang ajar', 'tidak berguna', 'sia sia',
            'percuma', 'menyebalkan', 'menjijikkan', 'memuakkan', 'pembenci'
        ]
    },

    'kata_kasar': {
        'weight': 3.5,  # SEDANG-TINGGI - kata kasar
        'keywords': [
            'anjing', 'asu', 'babi', 'monyet', 'kntl', 'mmk', 'bgst', 'bitch',
            'fuck', 'shit', 'damn', 'cok', 'tai', 'berak', 'kentut', 'bangke',
            'perek', 'lonte', 'pelacur', 'sundal', 'jalang'
        ]
    },

    'ujaran_kebencian': {
        'weight': 3.0,  # SEDANG - ujaran kebencian
        'keywords': [
            'benci', 'jahat', 'buruk', 'busuk', 'setan', 'iblis', 'dajal',
            'laknat', 'terkutuk', 'neraka', 'celaka', 'sialan', 'bangsat',
            'kafir', 'murtad', 'sesat', 'barbar', 'biadab'
        ]
    },

    'merendahkan': {
        'weight': 2.5,  # SEDANG-RENDAH - merendahkan
        'keywords': [
            'gelandang', 'pengemis', 'licik', 'pembohong', 'penipu', 'korup',
            'primitif', 'kere', 'miskin', 'kampungan', 'ndeso', 'culun',
            'norak', 'katrok', 'gondrong', 'preman', 'berandal'
        ]
    },

    'ejekan_fisik': {
        'weight': 2.0,  # RENDAH - ejekan fisik
        'keywords': [
            'jelek', 'buruk rupa', 'cacat', 'tua', 'bangka', 'keriput', 'botak',
            'bau', 'kotor', 'dekil', 'lusuh', 'compang camping', 'rontok',
            'pendek', 'tinggi', 'gendut', 'kurus', 'hitam', 'item'
        ]
    }
}
print("=== CYBERBULLYING RULES DENGAN BOBOT ===")
for category, info in CYBERBULLYING_RULES.items():
    print(f"{category}: weight = {info['weight']}, {len(info['keywords'])} keywords")

=== CYBERBULLYING RULES DENGAN BOBOT ===
berbahaya: weight = 5.0, 20 keywords
serangan_langsung: weight = 4.0, 18 keywords
kata_kasar: weight = 3.5, 21 keywords
ujaran_kebencian: weight = 3.0, 18 keywords
merendahkan: weight = 2.5, 17 keywords
ejekan_fisik: weight = 2.0, 19 keywords


In [52]:
def analyze_cyberbullying_with_rules(text):
    if pd.isna(text) or not isinstance(text, str):
        return {
            'total_score': 0,
            'word_count': 0,
            'categories': {},
            'words_found': []
        }

    text_lower = text.lower()
    total_score = 0
    total_count = 0
    category_scores = {}
    words_found = []

    for category, info in CYBERBULLYING_RULES.items():
        category_score = 0
        category_count = 0

        for keyword in info['keywords']:
            pattern = r'\b' + re.escape(keyword) + r'\b'
            matches = re.findall(pattern, text_lower)
            count = len(matches)

            if count > 0:
                weighted_score = count * info['weight']

                category_score += weighted_score
                category_count += count
                total_score += weighted_score
                total_count += count

                words_found.append({
                    'keyword': keyword,
                    'category': category,
                    'count': count,
                    'weight': info['weight'],
                    'weighted_score': weighted_score
                })

        if category_score > 0:
            category_scores[category] = {
                'score': category_score,
                'count': category_count
            }

    return {
        'total_score': round(total_score, 2),
        'word_count': total_count,
        'categories': category_scores,
        'words_found': words_found
    }
# Test dengan beberapa contoh
print("🧪 TESTING RULE-BASED ANALYSIS:")
test_cases = [
    "kamu bodoh sekali",  # serangan_langsung: 4.0
    "anjing kamu menyebalkan",  # kata_kasar + serangan_langsung: 3.5 + 4.0 = 7.5
    "aku akan bunuh kamu bodoh",  # berbahaya + serangan_langsung: 5.0 + 4.0 = 9.0
    "kamu jelek banget",  # ejekan_fisik: 2.0
    "lantai",  # 0
]

for i, text in enumerate(test_cases, 1):
    result = analyze_cyberbullying_with_rules(text)
    print(f"\n{i}. '{text}'")
    print(f"   Score: {result['total_score']}")
    print(f"   Words: {result['word_count']}")
    if result['categories']:
        print(f"   Categories: {list(result['categories'].keys())}")
        for word in result['words_found']:
            print(f"     - {word['keyword']} ({word['category']}: {word['weighted_score']})")

🧪 TESTING RULE-BASED ANALYSIS:

1. 'kamu bodoh sekali'
   Score: 4.0
   Words: 1
   Categories: ['serangan_langsung']
     - bodoh (serangan_langsung: 4.0)

2. 'anjing kamu menyebalkan'
   Score: 7.5
   Words: 2
   Categories: ['serangan_langsung', 'kata_kasar']
     - menyebalkan (serangan_langsung: 4.0)
     - anjing (kata_kasar: 3.5)

3. 'aku akan bunuh kamu bodoh'
   Score: 9.0
   Words: 2
   Categories: ['berbahaya', 'serangan_langsung']
     - bunuh (berbahaya: 5.0)
     - bodoh (serangan_langsung: 4.0)

4. 'kamu jelek banget'
   Score: 2.0
   Words: 1
   Categories: ['ejekan_fisik']
     - jelek (ejekan_fisik: 2.0)

5. 'lantai'
   Score: 0
   Words: 0


In [53]:
print("🚀 APPLYING RULE-BASED ANALYSIS TO ENTIRE DATASET...")

results = []
scores = []
word_counts = []

for idx, text in enumerate(data['processed_text']):
    if idx % 1000 == 0 and idx > 0:
        print(f"   Processed {idx:,}/{len(data):,} records...")

    result = analyze_cyberbullying_with_rules(text)
    results.append(result)
    scores.append(result['total_score'])
    word_counts.append(result['word_count'])

data['cyberbullying_score'] = scores
data['cyberbullying_word_count'] = word_counts

print("✅ Rule-based analysis completed!")

print(f"\n📊 BASIC STATISTICS:")
print(f"   Total texts: {len(data):,}")
print(f"   Texts with cyberbullying: {sum(s > 0 for s in scores):,} ({np.mean([s > 0 for s in scores]) * 100:.1f}%)")
print(f"   Score range: {min(scores):.1f} - {max(scores):.1f}")
print(f"   Average score (all): {np.mean(scores):.2f}")
print(f"   Average score (cyberbullying only): {np.mean([s for s in scores if s > 0]):.2f}")

🚀 APPLYING RULE-BASED ANALYSIS TO ENTIRE DATASET...
   Processed 1,000/18,775 records...
   Processed 2,000/18,775 records...
   Processed 3,000/18,775 records...
   Processed 4,000/18,775 records...
   Processed 5,000/18,775 records...
   Processed 6,000/18,775 records...
   Processed 7,000/18,775 records...
   Processed 8,000/18,775 records...
   Processed 9,000/18,775 records...
   Processed 10,000/18,775 records...
   Processed 11,000/18,775 records...
   Processed 12,000/18,775 records...
   Processed 13,000/18,775 records...
   Processed 14,000/18,775 records...
   Processed 15,000/18,775 records...
   Processed 16,000/18,775 records...
   Processed 17,000/18,775 records...
   Processed 18,000/18,775 records...
✅ Rule-based analysis completed!

📊 BASIC STATISTICS:
   Total texts: 18,775
   Texts with cyberbullying: 7,835 (41.7%)
   Score range: 0.0 - 119.5
   Average score (all): 1.83
   Average score (cyberbullying only): 4.40


In [54]:
data

,original_text,processed_text,cyberbullying_score,cyberbullying_word_count
0,Mending plonga plong Tinimbang pinter go minteri,mending bingung plong timbang pintar go minteri,0.0,0
1,Hidup amin Raiso bapa repormasi,hidup raiso bapa repormasi,0.0,0
2,Mantaappps... Btlllll pak...,mantaps btl,0.0,0
3,Yang menghina itu lambemu min,menghina itu lambe min,0.0,0
4,Amin rais manusia paling redah pikiranya..peng...,manusia redah pikiranya abdi hawa nafsu iri de...,0.0,0
...,...,...,...,...
18770,@kompascom Tapi aiman bertanya ke pengacaranya...,aiman acara ijazah edar pegang arti ini lindun...,0.0,0
18771,@muannas_alaidid Yg jelas omongannya Jokowi pa...,omong palsu titik,0.0,0
18772,@CakKhum depresi si om Jokowi,depresi si om,0.0,0
18773,@TriSartopo @ch_chotimah2 @jokowi Mana berani ...,berani tatut hm maklum curut berani cuap aja y...,0.0,0


In [55]:
def create_multiclass_labels(scores):
    labels = np.zeros(len(scores), dtype=int)
    labels[np.array(scores) == 0] = 0
    labels[(np.array(scores) > 0) & (np.array(scores) <= 3.0)] = 1
    labels[(np.array(scores) > 3.0) & (np.array(scores) <= 8.0)] = 2
    labels[np.array(scores) > 8.0] = 3

    return labels

data['label'] = create_multiclass_labels(data['cyberbullying_score'])

LABEL_DESCRIPTIONS = {
    0: "Tidak ada Cyberbullying",
    1: "Tingkat Keparahan Rendah",
    2: "Tingkat Keparahan Sedang",
    3: "Tingkat Keparahan Tinggi"
}

print("🏷️ MULTI-CLASS LABELING RESULTS:")
print("=" * 50)

label_counts = data['label'].value_counts().sort_index()
total_samples = len(data)

for label in range(4):
    count = label_counts.get(label, 0)
    percentage = (count / total_samples) * 100
    class_scores = data[data['label'] == label]['cyberbullying_score']
    min_score = class_scores.min() if len(class_scores) > 0 else 0
    max_score = class_scores.max() if len(class_scores) > 0 else 0
    avg_score = class_scores.mean() if len(class_scores) > 0 else 0
    print(f"\n📊 Class {label} - {LABEL_DESCRIPTIONS[label]}:")
    print(f"   Count: {count:,} ({percentage:.1f}%)")
    print(f"   Score range: [{min_score:.1f}, {max_score:.1f}]")
    print(f"   Average score: {avg_score:.2f}")

print(f"\n🔍 CATEGORY ANALYSIS BY CLASS:")
for label in range(4):
    print(f"\n--- CLASS {label}: {LABEL_DESCRIPTIONS[label]} ---")
    class_results = [results[i] for i in range(len(results)) if data.iloc[i]['label'] == label]

    if label == 0:
        print("   No cyberbullying categories (as expected)")
        continue

    category_counts = defaultdict(int)
    category_scores = defaultdict(float)

    for result in class_results:
        for category, info in result['categories'].items():
            category_counts[category] += 1
            category_scores[category] += info['score']

    if category_counts:
        print("   Top categories:")
        sorted_cats = sorted(category_counts.items(), key=lambda x: x[1], reverse=True)
        for cat, count in sorted_cats:
            avg_score = category_scores[cat] / count if count > 0 else 0
            print(f"     • {cat}: {count} texts (avg score: {avg_score:.1f})")
    else:
        print("   No categories found")

🏷️ MULTI-CLASS LABELING RESULTS:

📊 Class 0 - Tidak ada Cyberbullying:
   Count: 10,940 (58.3%)
   Score range: [0.0, 0.0]
   Average score: 0.00

📊 Class 1 - Tingkat Keparahan Rendah:
   Count: 3,698 (19.7%)
   Score range: [2.0, 3.0]
   Average score: 2.36

📊 Class 2 - Tingkat Keparahan Sedang:
   Count: 3,509 (18.7%)
   Score range: [3.5, 8.0]
   Average score: 5.11

📊 Class 3 - Tingkat Keparahan Tinggi:
   Count: 628 (3.3%)
   Score range: [8.5, 119.5]
   Average score: 12.39

🔍 CATEGORY ANALYSIS BY CLASS:

--- CLASS 0: Tidak ada Cyberbullying ---
   No cyberbullying categories (as expected)

--- CLASS 1: Tingkat Keparahan Rendah ---
   Top categories:
     • ejekan_fisik: 1987 texts (avg score: 2.0)
     • ujaran_kebencian: 956 texts (avg score: 3.0)
     • merendahkan: 755 texts (avg score: 2.5)

--- CLASS 2: Tingkat Keparahan Sedang ---
   Top categories:
     • serangan_langsung: 1359 texts (avg score: 4.4)
     • ejekan_fisik: 1272 texts (avg score: 3.1)
     • ujaran_kebencia

In [56]:
data

,original_text,processed_text,cyberbullying_score,cyberbullying_word_count,label
0,Mending plonga plong Tinimbang pinter go minteri,mending bingung plong timbang pintar go minteri,0.0,0,0
1,Hidup amin Raiso bapa repormasi,hidup raiso bapa repormasi,0.0,0,0
2,Mantaappps... Btlllll pak...,mantaps btl,0.0,0,0
3,Yang menghina itu lambemu min,menghina itu lambe min,0.0,0,0
4,Amin rais manusia paling redah pikiranya..peng...,manusia redah pikiranya abdi hawa nafsu iri de...,0.0,0,0
...,...,...,...,...,...
18770,@kompascom Tapi aiman bertanya ke pengacaranya...,aiman acara ijazah edar pegang arti ini lindun...,0.0,0,0
18771,@muannas_alaidid Yg jelas omongannya Jokowi pa...,omong palsu titik,0.0,0,0
18772,@CakKhum depresi si om Jokowi,depresi si om,0.0,0,0
18773,@TriSartopo @ch_chotimah2 @jokowi Mana berani ...,berani tatut hm maklum curut berani cuap aja y...,0.0,0,0


In [57]:
print("📝 DETAILED EXAMPLES PER CLASS:")
print("=" * 80)

np.random.seed(42)

for label in range(4):
    print(f"\n🏷️ CLASS {label}: {LABEL_DESCRIPTIONS[label]}")
    print("-" * 70)

    class_data = data[data['label'] == label]
    if len(class_data) == 0:
        print("   No examples found")
        continue
    if len(class_data) >= 3:
        class_sorted = class_data.sort_values('cyberbullying_score')
        examples_idx = [
            class_sorted.index[0],
            class_sorted.index[len(class_sorted) // 2],
            class_sorted.index[-1]
        ]
        examples = data.loc[examples_idx]
    else:
        examples = class_data.sample(min(3, len(class_data)))

    for i, (idx, row) in enumerate(examples.iterrows(), 1):
        result = results[idx]
        print(f"\n   Example {i}:")
        print(f"   Score: {result['total_score']:.1f}")
        print(f"   Original: {row['original_text'][:80]}...")
        print(f"   Processed: {row['processed_text']}")

        if result['categories']:
            print(f"   Categories detected:")
            for category, info in result['categories'].items():
                weight = CYBERBULLYING_RULES[category]['weight']
                print(f"     • {category}: {info['score']:.1f} (weight {weight} × {info['count']} words)")

            print(f"   Words found: {[w['keyword'] for w in result['words_found']]}")
        else:
            print(f"   No cyberbullying words detected")

        print("   " + "-" * 65)

📝 DETAILED EXAMPLES PER CLASS:

🏷️ CLASS 0: Tidak ada Cyberbullying
----------------------------------------------------------------------

   Example 1:
   Score: 0.0
   Original: Mending plonga plong Tinimbang pinter go minteri...
   Processed: mending bingung plong timbang pintar go minteri
   No cyberbullying words detected
   -----------------------------------------------------------------

   Example 2:
   Score: 0.0
   Original: Adanya jokowi jln enak muka tau amin Raiso enga jadi presiden...
   Processed: jln enak muka raiso enga presiden
   No cyberbullying words detected
   -----------------------------------------------------------------

   Example 3:
   Score: 0.0
   Original: @powerangerpink7 @sarwonobs @DokterTifa @jokowi roy bedain panci rumah ama panci...
   Processed: roy bedain panci rumah ama panci kantor aja yakali bedain ijazah
   No cyberbullying words detected
   -----------------------------------------------------------------

🏷️ CLASS 1: Tingkat Keparahan Re

In [58]:
data.to_csv('data_preprocessed_with_labels.csv', index=False)